# グラフ
グラフの表現は配列で持たせたりmapで持たせたりできるが、配列はメモリ喰うので個人的にはmapがいいと思う


## 二部グラフ
https://qiita.com/drken/items/7f98315b56c95a6181a4
https://qiita.com/drken/items/e805e3f514acceb87602

あるノードの色というか値を決める→隣が決まる　を繰り返していく。グラフが連結ならDFSでおｋだが、そうでないならノードごとに見ていくなど。

In [ ]:
N = int(input())

color = [0] * N

# esに辺の情報が入っているものとする 
# vがnode, cが色
def dfs(v, c):
    color[v] = c
    # aは行先のノード、bは重み
    for a,b in es[v]:
        # 隣接しているのが同じ色ならアウト
        if color[v] == color[a]:
            return False
        # 隣接しているノードにまだ色が塗られてない場合に、-cを塗って矛盾が生じるならアウト
        if color[a] == 0 and !dfs(a, -c):
            return False
        
    return True
    
# この関数をノードの分だけforで回すとかで連結じゃなくても対応できる

# 最短経路問題

## ベルマンフォード法
**始点を固定している場合に使う**  
**グラフに負のコストの閉路がない場合使える**  
**逆に言うと負の閉路判定にも使える**
**辺について注目した手法？**  

d(i)：startからiまで行くのにかかるコスト  
と表すと、j->iなる辺があるときに、
**d(i) = min(d(j)+j->iのコスト, d(i))**  
これはd(j)までだどりつける場合を前提にしてるので、スタート地点からスタート地点までのコストを０と置いて、そこから辺を近い順にみていって、スタートに近い部分のコストから明らかになっていく

In [ ]:
# ノード数、辺、スタートのノード番号
d = [INF]*N # 書いてないので本来はINFの定義を忘れずにする
def BellmanFord(N,es,start):
    
    for a in range(N):
        # a->bにコストcで行く辺
        for b,c in es[a]:
            if d[a] != INF and d[a] + c < d[b]:
                d[b] = d[a] + c
                # 負の閉路がないなら、多くともすべてのノードを一回ずつ訪問（N-1回移動）すればゴールにつく
                # なので、N回目の移動で更新されるなら、負の閉路がある
                if a == n-1:
                    return INF
                
    return d[-1]
                
        
            

In [ ]:
"""

AtCoder Beginner Contest 061 D
https://atcoder.jp/contests/abc061/tasks/abc061_d

"""

N, M = map(int, input().split())

es = []
for i in range(M):
    a,b,c = map(int, input().split())
    es.append((a-1, b-1, c))

INF = float("inf")
d = [INF] * N
d[0] = 0
"""
最大経路探索的な問題
よくあるベルマンフォード法なら、コストが正の値で、負の閉路が見つかったら失敗、という形で最小経路を求めることになる。
今回は最長経路なので、正の閉路が求まったら失敗にしたい。
なので
・コストを負の値でもつ。表示するときに正に変える
・コストを負にするので閉路を見つけられる
みたいな形にする
"""

def solve(start):
    for i in range(N):
        for a,b,c, in es:
            if d[a] != INF and d[a] + (-c) < d[b]:
                d[b] = d[a] + (-c)
                # 最後まで確認しても更新されたら閉路あり
                if i == N-1 and b == N-1:
                    return "inf"

    return -1*d[-1]

print(solve(0))



In [ ]:
"""
AtCoder Beginner Contest 137 E
https://atcoder.jp/contests/abc137/tasks/abc137_e
"""
N,M,P = map(int, input().split())

es = []

for i in range(M):
    a,b,c = map(int, input().split())
    es.append((a-1, b-1, c))

INF = float("inf")

d = [INF for _ in range(N)] 
d[0] = 0
"""
頂点Nに向かうまでに関係ない閉路に注意する

無限にコインを拾える場合、
・ある閉路を一周したときの得点と支払いの収支がプラス
・その閉路からゴールまで行ける
である。これがない場合、全てのノードを経由してゴールまで行く（N-1回移動）までにコインの枚数が最大になる
なので、一回ベルマンフォード法をやった後もう一回行って一回目と２回目でゴールまでのコイン枚数に変化があればゴ、
ールに行くまでの道に無限にコインをとれる閉路があるといえる

"""

def solve(start, f):
    for i in range(N):
        for a,b,c, in es:
            if d[a] != INF and d[a] + (-c) + P < d[b]:
                # ２週目と区別する。２週目で更新される部分は-INFとか適当に置いておくと、ゴールまでの影響がわかる
                if f == 0:
                    d[b] = d[a] + (-c) + P
                else:
                    d[b] = -INF


    return d[-1]


first=solve(0,0)
second=solve(0, 1)
if first != second:
    print(-1)
else:
    print(max(-first, 0))




## ダイクストラ法

ベルマンフォード法は　ノードの数 x 辺の数　の計算を行う。一回の計算である点への最短距離が求まらないことから、無駄な計算が割と多い。


①　最短距離が確定した点と隣接する点を更新する  
②　①で使った最短距離が確定した点は２度と使わない

という方針で進める。

流れとして、

(i)始点は最短距離が０  
(ii)始点に隣接する点A,Bについて最短距離を更新する
(iii)Aに隣接する点B,Cについて～～

を繰り返す。ちなみに負の辺があるとだめ

In [ ]:
import heapq

INF = float("inf")
d = [INF] * N
d[0] = 0

es = []
# es[a]=(c,b) で、aからbへコストcみたいなのが詰まっていると想定


def dijkstra(start):
    
    q = []
    heapq.heappush(q,(0, start))
    while q:
        cost, a = heapq.heappop(q)
        # 現時点での最短経路の方がキューに入っているコストより小さいなら飛ばす
        if d[a] < cost:
            continue
        # ノードaにつながる辺についてみる
        for c,b in es[a]:
            # aからbに行くのが現時点でのbまでの最短経路のコストより小さい場合
            if d[a] + c < d[b]:
                # 更新してヒープに　bまでd[b]　を積む
                d[b] = d[a] + c
                heapq.heappush(q, (d[b], b))
                
"""
最後まで確認すると最短経路が求まる
計算量はO(辺の数 log ノードの数)

"""
            

In [ ]:
"""
At Coder ABC-35 D
https://atcoder.jp/contests/abc035/tasks/abc035_d
"""

import heapq

N,M,T = map(int, input().split())


A = list(map(int, input().split()))


"""

問題文から明らかに単一始点最短経路問題
制限的にベルマンフォードは無理。辺に負のコストがないからダイクストラの適用を考える
始点から別ノードは普通にダイクストラ。戻りの経路は？
→辺を逆にたどるダイクストラで戻りの道も分かる
移動にかかる時間以外をある１ノードに滞在するのにつかうとよい

** 有向グラフでも、戻りのコストも知りたいみたいな問題なら向きを逆にしてやると求められる

"""

es = [[] for _ in range(N)]
rev_es = [[] for _ in range(N)]
for i in range(M):
    a,b,c = map(int, input().split())
    es[a-1].append((c, b-1))
    rev_es[b-1].append((c, a-1))

INF = float("inf")
d = [INF] * N
d[0] = 0

# かくノードまで行く道のコストを求める
q = []
heapq.heappush(q, (0, 0))
while q:
    cost, a = heapq.heappop(q)
    # 現時点で明らかになってる最小コストの方がヒープの候補より小さいならパス
    if d[a] < cost:
        continue
    for c,b in es[a]:
        # a経由でbに行く方が現時点でのbへの最小コストより小さいなら更新
        if d[a] + c < d[b]:
            d[b] = d[a] + c
            # ヒープに「bまでd[b]で行けますよ」をつむ
            heapq.heappush(q, (d[b], b))

# 戻りのコストを求める
rev_d = [INF] * N
rev_d[0] = 0
q = []
heapq.heappush(q, (0, 0))
while q:
    cost, a = heapq.heappop(q)
    if rev_d[a] < cost:
        continue
    for c,b in rev_es[a]:
        if rev_d[a] + c < rev_d[b]:
            rev_d[b] = rev_d[a] + c
            heapq.heappush(q, (rev_d[b], b))

point = [0] * N
for i in range(N):
    point[i] = (T - d[i] - rev_d[i]) * A[i]

print(max(point))

In [ ]:
"""
AtCoder Beginner Contest 051 D
"""

import heapq

N,M = map(int, input().split())
es = [[] for _ in range(N)]

for i in range(M):
    a,b,c = map(int, input().split())
    es[a-1].append((c, b-1))
    es[b-1].append((c, a-1))


"""
・ノード数が少ないので、全ノード間の最短距離を求めても間に合う
・ある辺eがいずれかの最短距離に含まれる　→　その辺eが結ぶ２ノードの最短距離はeの距離（それぞれの最短距離を束ねてある２ノードの最短距離を作るから）
"""

INF = float("inf")
# 全ノード間の最短距離を求める
d = [[INF for _ in range(N)] for _ in range(N)]

for i in range(N):
    q = []
    d[i][i] = 0
    heapq.heappush(q, (0, i))
    while q:
        cost, a = heapq.heappop(q)
        if d[i][a] < cost:
            continue
        for c,b in es[a]:
            if d[i][a] + c < d[i][b]:
                d[i][b] = d[i][a] + c
                heapq.heappush(q, (d[i][b], b))


# 全ての辺について、その辺の間の距離cがa->bまでの最短距離か確かめる
# ある辺がいずれかのノード間の最短経路に含まれるなら、その辺（が結ぶノード）も最短距離になるから。
cnt = 0
for i in range(N):
    for c, b in es[i]:
        if d[i][b] != c:
            cnt += 1

# 無向グラフなので２方向分カウントされてるから２で割る
print(cnt//2)

## ワーシャルフロイド法

全ての点の間の最短距離を求める  
i->jの最短距離が知りたいとき、あるkがあるとしてi->jに行く方法は、i->k->jか、kを介さず行くか。  
これを全i,j,kについて実施する。計算量は多いが、実装は簡単なので単一始点でも使う機会はあるかもしれない

In [ ]:

for k in range(N):
    for i in range(N):
        for j in range(N):
            dp[i][j]  min(dp[i][j], do[i][k] + dp[k][j])

## 経路復元

今まで最短経路更新の時に  
if d(a) + c < d(b):  
    d(b) = d(a) + c  
のような形で更新してきた。そのままだが、これはbに行くときの最短経路はaに行ってからコストがcのa->bの辺をたどるということである。  
なので、目的地bの一つ手前がaであるから、上の式を満たすaをたどっていくと最短経路がゴールの方から求まる。  



あるいは、上の式で最短距離が更新されたときに、直前のノードaを配列などに記録しておくのもあり。

In [ ]:
b = N-1
route = [b]
while b != (startのノード)
    for c, a in (bに行く辺):
        # 下の条件をみたすノードaが、直前のノード
        if d[b] = d[a] + c:
            # 今度はaの直前を探すためにbに入れる。最終的にbにはスタートのノードが入る
            b = a
            route.append(a)
            
# ゴールからスタートに戻っていく順で入ってるので逆にして経路が完成
route = route[::-1]

## 最小全域木

全域木：無向グラフで、任意の２点間が繋がってるやつ  
最小全域木：↑の辺のコストの和が最小になるやつ（コストの大きいいらない辺を落とす）  

最小全域木を求める方法で典型的なのはクラスカル法とプリム法がある

## プリム法

ダイクストラ法風味。  
ある一つの頂点について、そいつとほかの頂点を結ぶ最小コストの辺を採用していく   
最小かコストら選んでいく→priority queueと相性がいい  


In [ ]:
import heapq

INF = float("inf")
# 始点０からの最小コスト
mins = [INF] * N
mins[0] = 0
# あるノードが木に入っているか
used = [False] * N
totalcost = 0
# 最初木を構成するノードは始点の０のみ
# 木に含まれない頂点のうち、木を構成するノードと繋がって居るもので最小のコストの辺を取り出す
q = []
for c,b in es[0]:
    heapq.heappush(q, (c,b))
def prim():
    while len(q) > 0:
        # 最小コストの辺を取り出す
        c,b = heapq.heappop(q)
        # 取り出した最小コストの辺が木に含まれていたらスルー
        if used[b] == True:
            continue

        # 取り出したbを木に取り込む
        used[b] = True
        for nc, nb in es[b]:
            # nはnext的な意味で使ってる
            heapq.heappush(q, (nc, nb))
        # トータルのコストにbに至る辺のコストを加算
        totalcost += c
    
    return totalcost 


    

## クラスカル法

コストの小さい順に辺を見ていき、閉路ができなければ木に取り込む。  
コストの小さい順に辺をみる→最小全域木なので  
閉路ができたらだめ→あるノードに至るのに最小の経路とそうでない経路ができるので最小全域木でなくなるから  

閉路ができるかどうか
aとbを繋ぐことを考える  
閉路ができない→aとbは同じ連結成分（部分木的な）に属しておらず、現在注目している辺を採用したときにはじめて同じ連結成分になる  
閉路ができる→aとbがもとから同じ連結成分に属していて、現在注目している辺を用いないでも繋がって居る場合  



同じ連結成分かどうかはUnionFindで求めればよい

In [ ]:
# UnionFindのコードは省略するので雰囲気
import heapq

# すべての辺についてコストの小さい順に並べる
# a->bでcのコスト。無向グラフなのでたぶん本当は逆向きも入れるべきかも？
q = []
for i in range(N):
    for c,b in es[i]:
        heapq.heappush(q, (c,a,b))
        
# UnionFind初期化
uf = UnionFind(N)

def kruskal():
    totalcost = 0
    # すべての辺について調べる
    for i in range(len(q)):
        # 辺をコストの小さい順に取り出す
        c,a,b = heapq.heappop(q)
        # a,bが同じUnionでない＝連結成分でないなら、木に取り込む
        if not uf.is_same(a,b):
            uf.unite(a,b)
            totalcost += c
            
    return totalcost

In [4]:
"""
p102 POJ 3255のやつ

２番目の最短経路を求める
bに向かう際の２番目の最短経路を考える。これも辺を軸として見ていく
bまでの二番目の最短経路は、
・ほかの頂点aまでの最短経路に辺a->bをつないだもの
・aまでの２番目の最短経路に辺a->bを繋いだもの
のどちらか。
なので、いつもならdという始点からの最短距離を保存する配列を用意していたが、
今回は２番目の最短経路も欲しいので　d1, d2 を用意してやるとよい
"""
import heapq

N,R = map(int, input().split())

es = [[] for _ in range(N)]
for i in range(R):
    a,b,c = map(int, input().split())
    es[a-1].append((c,b-1))
    es[b-1].append((c,a-1))
    
INF = float("inf")
d1 = [INF] * N
d1[0] = 0
d2 = [INF] * N

q = []
heapq.heappush(q, (0, 0))
while len(q) > 0:
    cost,a = heapq.heappop(q)
    # 現在分かっているaまでの２番目の最短距離のほうが、新たな候補より小さければスルー
    if d2[a] < cost:
        continue
    for c,b in es[a]:
        # aまでのコスト+a->bのコストが、現時点で分かっているbまでのコストより小さい場合更新する
        if d1[b] > cost + c:
            d1[b] = cost + c
            # 「bまでd1[b]で行くのが最小コストですよ」をキューに入れる
            heapq.heappush(q, (d1[b], b))
        # aまでのコスト+a->bのコストが、bまでの２番目の最小コストよりは小さいが最小コストよりは大きい場合
        if d2[b] > cost + c and d1[b] < cost + c:
            d2[b] = cost + c
            # ２番目の情報も残したいのでこれ
            heapq.heappush(q, (d2[b], b))
            
            
            
print(d2[-1])

4 4
1 2 100
2 3 250
2 4 200
3 4 100
450


In [15]:
"""
p103 POJ 3723

親密度が高い人がすでに徴兵されていると費用が安くなる
→親密度が高い関係から取り込んでいく
"""
#久々にUnionFindも書いてみる
class UnionFind:
    def __init__(self, N):
        self.parent = [i for i in range(N)]
        self.rank = [0] * N # 最初はそれぞれのノードが誰ともUnionをつくってないから木の深さが０
        self.count = 0 # unite回数
    
    # 根を返す
    def root(self, a):
        if self.parent[a] == a:
            return a
        else:
            # 親を見に行く
            self.parent[a] = self.root(self.parent[a])
            return self.parent[a]
    
    # 同じUnionに属するかどうか
    def is_same(self, a, b):
        return self.root(a) == self.root(b)
    
    # 同じUnionにする
    def unite(self, a, b):
        ra = self.root(a)
        rb = self.root(b)
        if ra == rb:
            return 
        if self.rank[ra] < self.rank[rb]:
            self.parent[ra] = rb
        else:
            self.parent[rb] = ra
            if self.rank[ra] == self.rank[rb]:
                self.rank[ra] += 1
        self.count += 1
            






import heapq

N,M,R = map(int, input().split())

node_num = N+M
es = []
for i in range(R):
    x,y,d = map(int, input().split())
    es.append((-d, x, N+y))
    
q = []
for c,a,b in es:
    heapq.heappush(q, (c,a,b))

uf = UnionFind(node_num)
def kruskal():
    totalcost = 0
    #すべての辺についてみていく
    while len(q) > 0:
        c,b,a = heapq.heappop(q)
        if not uf.is_same(a,b):
            uf.unite(a,b)
            totalcost += -c
    return totalcost

saved = kruskal()
print((N+M)*10000 - saved)


"""


5 5 8
4 3 6831
1 3 4583
0 0 6592
0 1 3063
3 3 4975
1 3 2049
4 2 2104
2 2 781
71071
"""

5 5 8
4 3 6831
1 3 4583
0 0 6592
0 1 3063
3 3 4975
1 3 2049
4 2 2104
2 2 781
71071


'\n\n\n5 5 8\n4 3 6831\n1 3 4583\n0 0 6592\n0 1 3063\n3 3 4975\n1 3 2049\n4 2 2104\n2 2 781\n71071\n'

In [ ]:
import heapq
class UnionFind:
    def __init__(self, N):
        self.parent = [i for i in range(N)]
        self.rank = [0] * N # 最初はそれぞれのノードが誰ともUnionをつくってないから木の深さが０
        self.count = 0 # unite回数
    
    # 根を返す
    def root(self, a):
        if self.parent[a] == a:
            return a
        else:
            # 親を見に行く
            self.parent[a] = self.root(self.parent[a])
            return self.parent[a]
    
    # 同じUnionに属するかどうか
    def is_same(self, a, b):
        return self.root(a) == self.root(b)
    
    # 同じUnionにする
    def unite(self, a, b):
        ra = self.root(a)
        rb = self.root(b)
        if ra == rb:
            return 
        if self.rank[ra] < self.rank[rb]:
            self.parent[ra] = rb
        else:
            self.parent[rb] = ra
            if self.rank[ra] == self.rank[rb]:
                self.rank[ra] += 1
        self.count += 1

N = int(input())

cities = [tuple(map(int, input().split())) for _ in range(N)]

"""
AtCoder Beginner Contest 065 D
あるノードについてx座標の差とや座標の差それぞれの絶対値の小さいほうがコストになる
最小全域木を作ることでコストを最小にする
なので、
・x座標が小さい順で並べ替えたノードリスト
・ｙ座標が小さい順で並べ替えたノードリスト
を作って、それらのノード間に辺が張られているとする。
これで最小全域木ができるまでクラスカル法とかで頑張る
（ｘ座標とy座標のそれぞれについて近い順で辺が張られているとして、いらないものを落としていく形）

愚直に考えると、全ノード間に辺が張られているばあを考えて、コストの小さい順に～とできそうだが、N^2ぐらいの辺ができるので間に合わない。
今回はxとyの座標の差の小さい方をコストにするという話なので、それらを軸に辺を作った。
で、小さい順（別に大きい順でも構わないが）にソートしたのは、ソートしておけばインデックスの隣接しているもののコストが最小になるから。
"""
xs = []
ys = []
for i,(x,y) in enumerate(cities):
    xs.append((x, i))
    ys.append((y, i))

xs.sort()
ys.sort()

es = []
for (xa, a), (xb,b) in zip(xs, xs[1:]):
    es.append((abs(xa-xb), a, b))

for (ya, a), (yb, b) in zip(ys, ys[1:]):
    es.append((abs(ya-yb), a, b))

es.sort()

uf = UnionFind(N)

totalcost = 0
"""
esは辺のコストが小さい順で並んでいるので、コストが小さい順に繋がれる
また、esは2N個なので、十分間に合う
"""
for c, a, b in es:
    if uf.is_same(a,b):
        continue
    uf.unite(a,b)
    totalcost += c

print(totalcost)